In [16]:
import sys 
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import numpy as np


In [17]:
columns_to_sum = ['test_loss', 'test_loss_true', 'test_mae', 'test_mse']
clusters = ['WEBSTER', 'HIGH', 'TED', 'HAMILTON', 'RINCONADA', 'CAMBRIDGE', 'MPL', 'BRYANT']
data_path = "../data/LossCSVs"

In [34]:
import os

def load_csv(model, lead, strategy, censor_level):
    base_name = f"{data_path}/loss_{model}lead{lead}_{strategy}{censor_level}"

    aware_path = f"{base_name}.csv"
    unaware_path = f"{base_name}_unaware.csv"

    if not os.path.exists(aware_path):
        print(f"Warning: Missing data for aware {model} {lead} {strategy}{censor_level}.Path {aware_path} does not exist")
        raise FileNotFoundError
    if not os.path.exists(unaware_path):
        print(f"Warning: Missing data for unaware {model} {lead} {strategy}{censor_level}.Path {unaware_path} does not exist")
        raise FileNotFoundError

    aware_df = pd.read_csv(aware_path)
    unaware_df = pd.read_csv(unaware_path)
    return aware_df[aware_df.State != 'crashed'], unaware_df[unaware_df.State != 'crashed']

def calculate_losses(df, df_unaware, model_name, columns_to_sum):
    agg_mean = np.round(df[columns_to_sum].mean(),2)
    agg_std = np.round(df[columns_to_sum].std(),2)

    agg_mean_unaware = np.round(df_unaware[columns_to_sum].mean(),2)
    agg_std_unaware = np.round(df_unaware[columns_to_sum].std(),2)

    losses = [model_name.upper()]
    for col in columns_to_sum:
        # Unaware models don't have a censored loss
        if col == 'test_loss':
            losses.append('$' + agg_mean[col].astype(str) + r' \pm ' + agg_std[col].astype(str) + '$ / -')
        else:
            losses.append('$' + agg_mean[col].astype(str) + r' \pm ' + agg_std[col].astype(str) + '$ / $' + agg_mean_unaware[col].astype(str) + r' \pm ' + agg_std_unaware[col].astype(str) + '$')
    return losses

leads = [1, 48]
strategies = {
    'dyn': [1, 2],
    'stat': [2, 3]
}

models = ['gru', 'lstm', 'ar']
leads = [1, 48]

# Store the dataframes in a dictionary

# Iterate over all the combinations
df_losses = pd.DataFrame(columns=['Forecast lead', 'Model', 'Strategy'] + columns_to_sum)

no_runs_dict = {}

for lead in leads:
    for strategy, censor_levels in strategies.items():
        for censor_level in censor_levels:
            for model in models:
                # Load data
                try:
                    aware, unaware = load_csv(model, lead, strategy, censor_level)
                except:
                    continue
                # Figure out how many runs we have
                no_runs_aware = np.floor(aware.shape[0]/8)
                no_runs_unaware = np.floor(unaware.shape[0]/8)
                print(f"Model {model} lead {lead} strategy {strategy} censor level {censor_level} has {no_runs_aware} aware runs and {no_runs_unaware} unaware runs")
                # Take the first no_runs runs
                aware = aware.groupby('cluster').head(no_runs_aware).copy()
                unaware = unaware.groupby('cluster').head(no_runs_unaware).copy()

                aware.loc[:, "run_no"] = np.repeat(np.arange(1, 1+no_runs_aware), 8)
                unaware.loc[:, "run_no"] = np.repeat(np.arange(1, 1+no_runs_unaware), 8)
                # Mean over runs
                unaware_agg = unaware.groupby('run_no').mean().reset_index()
                # Mean over runs
                aware_agg = aware.groupby('run_no').mean().reset_index()
                df_losses.loc[len(df_losses)] = [lead, strategy+ str(censor_level)] + calculate_losses(aware_agg, unaware_agg, model, columns_to_sum)

Model gru lead 1 strategy dyn censor level 1 has 4.0 aware runs and 4.0 unaware runs
Model gru lead 1 strategy dyn censor level 2 has 7.0 aware runs and 4.0 unaware runs
Model ar lead 1 strategy dyn censor level 2 has 7.0 aware runs and 2.0 unaware runs
Model gru lead 1 strategy stat censor level 2 has 4.0 aware runs and 3.0 unaware runs
Model gru lead 1 strategy stat censor level 3 has 4.0 aware runs and 4.0 unaware runs


In [35]:
print(df_losses.groupby(['Forecast lead', 'Model']).head().style.to_latex())

\begin{tabular}{lrllllll}
 & Forecast lead & Model & Strategy & test_loss & test_loss_true & test_mae & test_mse \\
0 & 1 & dyn1 & GRU & $0.88 \pm 0.0$ / - & $-0.16 \pm 0.01$ / $-0.14 \pm 0.0$ & $0.58 \pm 0.01$ / $0.45 \pm 0.01$ & $0.82 \pm 0.04$ / $0.48 \pm 0.01$ \\
1 & 1 & dyn2 & GRU & $0.82 \pm 0.01$ / - & $-0.15 \pm 0.01$ / $-0.07 \pm 0.0$ & $0.61 \pm 0.02$ / $0.41 \pm 0.01$ & $0.91 \pm 0.07$ / $0.41 \pm 0.01$ \\
2 & 1 & dyn2 & AR & $0.94 \pm 0.02$ / - & $0.09 \pm 0.07$ / $0.14 \pm 0.08$ & $1.07 \pm 0.12$ / $0.65 \pm 0.0$ & $3.23 \pm 0.95$ / $1.06 \pm 0.13$ \\
3 & 1 & stat2 & GRU & $0.46 \pm 0.01$ / - & $-0.09 \pm 0.02$ / $0.6 \pm 0.0$ & $0.75 \pm 0.02$ / $0.27 \pm 0.01$ & $1.41 \pm 0.1$ / $0.16 \pm 0.0$ \\
4 & 1 & stat3 & GRU & $0.59 \pm 0.0$ / - & $-0.13 \pm 0.01$ / $0.4 \pm 0.0$ & $0.66 \pm 0.02$ / $0.34 \pm 0.0$ & $1.12 \pm 0.08$ / $0.24 \pm 0.0$ \\
\end{tabular}



In [33]:
load_csv('gru', 1, 'dyn', 2)[1][columns_to_sum + ['censored']]

,test_loss,test_loss_true,test_mae,test_mse,censored
0,-0.055752,-0.055752,0.413095,0.344095,False
1,-0.451963,-0.451963,0.566749,0.686327,False
2,-0.090010,-0.090010,0.404755,0.425338,False
3,0.595000,0.595000,0.264380,0.152857,False
4,0.605711,0.605711,0.274736,0.169950,False
5,-0.899132,-0.899132,0.569284,0.721515,False
6,0.609711,0.609711,0.296753,0.183325,False
7,-0.875505,-0.875505,0.491127,0.575168,False
8,-0.053465,-0.053465,0.423689,0.353098,False
9,-0.459319,-0.459319,0.533218,0.621510,False
